# Capstone Project - The Battle of Neighborhoods

## Introduction/Business Problem

#### introduction where you discuss the business problem and who would be interested in this project.

### "Which district in 5 main economic center district in Bangkok new entrepreneur can open coffee shop by escape STARBUCK and Other Coffee Shop"

### List of 5 main economic center district in Bangkok
1. Sathon(เขตสาทร)  
1. Pathumwan(เขตปทุมวัน) 
1. Ratchathewi(เขตราชเทวี)  
1. Phaya Thai(เขตพญาไท) 
1. phra nakhon(เขตพระนคร)

### In the trend to drink coffee is coming so many people want to open new coffee shop but how they know where is district in bangkok suitable for open new business by not his competitor with world largest coffee company and exist coffee shop so i will use Foresquare venues category locations with local district location to  select 10 possible locations to build the coffee. Which location should be suggested to the new entrepreneur?

# Data

#### Data where you describe the data that will be used to solve the problem and the source of the data. <br/>

According to the question, following data to resolve the problem.



 1. Geographic coordinate of Bangkok and Disticct

i can find location of bangkok and district from  http://nominatim.openstreetmap.org/   then use  http://polygons.openstreetmap.fr/index.py to get polygon of district before create locations area before use 
Foresqure to get coffe venues in bangkok and plot to map

Bangkok district data https://nominatim.openstreetmap.org/details.php?place_id=198742921

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium # map rendering library
from shapely.geometry import Polygon
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [2]:
address = 'Bangkok'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangkok are {}, {}.'.format(latitude, longitude))

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Bangkok are 13.7538929, 100.8160803.


In [3]:
map_bangkok = folium.Map(location=[latitude, longitude], zoom_start=9)

### Map Bangkok

In [4]:
map_bangkok.choropleth(geo_data='datasets/bangkok_geojson.txt',fill_color=None, fill_opacity=0.1,name='กรุงเทพ', legend_name='กรุงเทพ')

In [5]:
import requests
import lxml.html as lh
import geocoder
import json
from flask import jsonify

### get all district of bangkok from list

In [6]:
import os
bangkok = pd.read_excel('datasets/BangkokGeoData.xlsx')
bangkok.head()

,Local name,OSM,Address rank,Admin level,Distance
0,วัฒนา,2938036,12,6,~22.9 km
1,เขตสาทร,3147425,12,6,~28.7 km
2,เขตดุสิต,92069,12,6,~30.5 km
3,เขตบางนา,3146412,12,6,~19.9 km
4,เขตบางแค,2946804,12,6,~42.4 km


### use OSM and API from Openstreet map for load data if not exist for fast load district next time

In [7]:
datadist = {}
for index, district in bangkok.iterrows():
    filename = 'datasets/GeoData/'+district['Local name']+'_data.json'
    exists = os.path.isfile(filename)
    if exists:
        #print('a')
        with open(filename) as f:
            geodata = json.load(f)
            map_bangkok.choropleth(geo_data=geodata['geojson'],fill_color='orange', fill_opacity=0.1,name=district['Local name'], legend_name=district['Local name'])
            datadist[district['Local name']] =geodata
    else:
        print('b')
        url = "https://nominatim.openstreetmap.org/reverse?format=json&osm_id={}&osm_type=R&polygon_geojson=1".format(district['OSM'])
        data =requests.get(url)
        geodata = json.loads(data.content)
        with open(filename, 'w') as outfile:
            json.dump(geodata, outfile)
        map_bangkok.choropleth(geo_data=geodata['geojson'],fill_color='orange', fill_opacity=0.1,name=district['Local name'], legend_name=district['Local name'])



In [8]:
map_bangkok

### Define Foursquare Credentials and Version

In [9]:
CLIENT_ID = 'IFQ0UJV14LTHXDLQV23VYM4L51UXD5PH2EWE3JFE4BGH5RQW' # your Foursquare ID
CLIENT_SECRET = 'G2KBU3MYPNZBWHTXJU10WN5RG2DIYQLP2X3DBOF1XQSJ5QJ3' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

###  With above data, I can build a content-based recommender systems to resolve the problem.

Combine with FourSquare API on counting how many different venues (Coffee Shop,Food, Transport) and Coffee shop list,

The weighted matrix can be applied on 5 possible distict locations with venues information to generate a ranking result. The the top one on the ranking list can be recommended to new entrepreneur .

## Methodology
Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, and what machine learnings were used and why.<br>
NEXT

## Results
Results section where you discuss the results <br>
NEXT

## Discussion
Discussion section where you discuss any observations you noted and any recommendations you can make based on the results <br>
NEXT

## Conclusion
Conclusion section where you conclude the report. <br>
NEXT

In [ ]:
# district_venues = getNearbyVenues(names='เขตวัฒนา',
#                                    latitudes=toronto_data['Latitude'],
#                                    longitudes=toronto_data['Longitude']
#                                   )